In [1]:
import pickle
import networkx as nx
import pandas as pd
import os
from utils import get_number_edges_to_show

dir_path = os.getcwd()
pcg_data_path = f"{dir_path}/result/nodes_and_edges_PCG.pkl"
lotus_south_data_path = f"{dir_path}/result/nodes_and_edges_lotus_south.pkl"
data_path_dict = {"PCG": pcg_data_path, "Lotus South": lotus_south_data_path}
data_path_dict = {"PCG": pcg_data_path}


def filter_non_arrow_edges(edges, slider_value):
    new_edges = []
    for edge in edges:
        if edge["data"]["arrow_weight"] != "none":
            if edge["data"]["raw_weight"] >= slider_value:
                new_edges.append(edge)
    return new_edges


debug_list = []
all_data_list = []
for company, data_path in data_path_dict.items():
    G = nx.DiGraph()
    data = pickle.load(open(data_path, "rb"))

    nodes = data[0]
    edges = data[1]
    line_weights = [edge["data"]["raw_weight"] for edge in edges]
    # num_edges_to_show = get_number_edges_to_show(len(nodes))
    num_edges_to_show = get_number_edges_to_show(100)
    sorted_weights = sorted(line_weights, reverse=True)
    # The threshold is the weight of the (num_edges_to_show)-th edge (0-indexed)
    slider_value = sorted_weights[num_edges_to_show - 1]
    filter_edges = filter_non_arrow_edges(edges, slider_value)
    for edge in filter_edges:
        G.add_edge(
            edge["data"]["source"],
            edge["data"]["target"],
            weight=edge["data"]["raw_weight"],
        )
        if edge["data"]["source"] == "risk_PCG_40":
            debug_list.append(edge)
    in_degree_centrality_dict = nx.in_degree_centrality(G)
    out_degree_centrality_dict = nx.out_degree_centrality(G)
    betweenness_dict_weight = nx.betweenness_centrality(G, weight="weight")
    betweenness_dict_non_weight = nx.betweenness_centrality(G)

    # create list of data so I can convert to dataframe later
    for node in nodes:
        row_data = {
            "company": company,
            "risk_id": node["data"]["id"],
            "risk_name": node["data"]["label"],
            "in_degree_centrality": in_degree_centrality_dict.get(
                node["data"]["id"], None
            ),
            "out_degree_centrality": out_degree_centrality_dict.get(
                node["data"]["id"], None
            ),
            "betweenness_centrality_weight": betweenness_dict_weight.get(
                node["data"]["id"], None
            ),
            "betweenness_centrality_non_weight": betweenness_dict_non_weight.get(
                node["data"]["id"], None
            ),
        }
        all_data_list.append(row_data)
# num_edges_to_show = get_number_edges_to_show(total_nodes)

# if num_edges_to_show == 0:
#     # If showing 0 edges, set threshold higher than max weight
#     slider_value = max(line_weights) + 1 if line_weights else float("inf")
# elif num_edges_to_show == total_edges:
#     # If showing all edges, set threshold lower than min weight
#     slider_value = min(line_weights) - 1 if line_weights else float("-inf")
# else:
#     # Sort weights descending and find the weight at the index corresponding to the number of edges
#     sorted_weights = sorted(line_weights, reverse=True)
#     # The threshold is the weight of the (num_edges_to_show)-th edge (0-indexed)
#     slider_value = sorted_weights[num_edges_to_show - 1]
all_data_df = pd.DataFrame(all_data_list)
all_data_df.head()

/Users/ford/Documents/coding_trae/cro_rmi_improvement_feature/src/cro_rmi_improvement_feature/plot_risk_related


,company,risk_id,risk_name,in_degree_centrality,out_degree_centrality,betweenness_centrality_weight,betweenness_centrality_non_weight
0,PCG,risk_PCG_0,Accounting errors,0.019231,0.000000,0.0,0.0
1,PCG,risk_PCG_1,Business interruption from fire hazards,0.000000,0.019231,0.0,0.0
2,PCG,risk_PCG_2,Business interruption from flood,0.019231,0.000000,0.0,0.0
3,PCG,risk_PCG_3,Business interruption from labor dispute,NaN,NaN,NaN,NaN
4,PCG,risk_PCG_4,Business interruption from natural disasters,0.000000,0.019231,0.0,0.0


In [2]:
len(debug_list)

11

In [3]:
len(filter_edges)

160